# Settings script

In [ ]:
MODEL_1 = True
MODEL_2 = False

RUN_RAYTUNE = True
USE_FULL_DATASET = False  # If false, use small dataset instead (10K pictures, 100 per class)

# Ensure exactly one model is selected
assert sum([MODEL_1, MODEL_2]) == 1, "Exactly one model must be selected."

# Imports

In [ ]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import pathlib

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from tqdm import tqdm

# RayTune imports (safe even if not used)
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

# Set GPU variable

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# Data paths

In [ ]:
ROOT = pathlib.Path().resolve()

if USE_FULL_DATASET:
    IMAGE_DIR = ROOT / "data" / "archive" / "food-101" / "food-101" / "images"
else:
    IMAGE_DIR = ROOT / "data" / "food-101-small"

print("Using IMAGE_DIR =", IMAGE_DIR)

if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Missing dataset folder:\n{IMAGE_DIR}")

# Safety: avoid __MACOSX folders
bad_paths = list((ROOT / "data" / "archive" / "food-101").rglob("__MACOSX"))
if bad_paths:
    print("⚠ WARNING: __MACOSX folders detected.")

print("Dataset path OK.")

# Data preprocessing

In [ ]:
img_size = 224

train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

full_dataset = datasets.ImageFolder(IMAGE_DIR, transform=train_transform, allow_empty=True)
class_names = full_dataset.classes
num_classes = len(class_names)

print("Total images:", len(full_dataset))
print("Classes:", num_classes)

# Split data into sets

In [ ]:
train_size = int(0.7 * len(full_dataset))
val_size   = int(0.2 * len(full_dataset))
test_size  = len(full_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoaders

In [ ]:
val_ds.dataset.transform = test_transform
test_ds.dataset.transform = test_transform

batch_size_default = 32

train_loader = DataLoader(train_ds, batch_size=batch_size_default, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size_default, shuffle=False, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=batch_size_default, shuffle=False)

# Build Model

In [ ]:
def build_model_1(lr, num_classes):
    """EfficientNet-B0 model"""
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    model = efficientnet_b0(weights=weights)

    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)

    for p in model.features.parameters():
        p.requires_grad = False

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    return model, optimizer, criterion


def build_model_2(lr, num_classes):
    """Placeholder model — intentionally blank"""
    raise NotImplementedError("MODEL_2 not implemented yet.")

In [ ]:
if MODEL_1:
    build_model = build_model_1
elif MODEL_2:
    build_model = build_model_2

# Define training and evaluation functions

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for imgs, labels in tqdm(loader, desc="Training", leave=False):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return total_loss / total, correct / total

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    return total_loss / total, correct / total

# Define Hyperparameter Optimization

In [ ]:
def tune_train(config):
    batch_size = config["batch_size"]
    lr = config["lr"]

    # local transforms
    train_ds.dataset.transform = train_transform
    val_ds.dataset.transform   = test_transform

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)

    model, optimizer, criterion = build_model(lr, num_classes)

    for epoch in range(config["epochs"]):
        train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        tune.report({"loss": float(val_loss), "accuracy": float(val_acc)})

# Run RayTune (If toggled on)

In [ ]:
if RUN_RAYTUNE:
    os.environ["RAY_DISABLE_METRICS_EXPORT"] = "1"

    search_space = {
        "lr": tune.loguniform(1e-5, 1e-2),
        "batch_size": tune.choice([16, 32, 64]),
        "epochs": 3
    }

    scheduler = ASHAScheduler(metric="accuracy", mode="max")
    reporter  = CLIReporter(metric_columns=["loss", "accuracy"])

    tuner = tune.Tuner(
        tune.with_resources(
            tune_train,
            resources={"cpu": 4, "gpu": 1 if torch.cuda.is_available() else 0},
        ),
        param_space=search_space,
        tune_config=tune.TuneConfig(
            scheduler=scheduler,
            num_samples=6,
        ),
        run_config=tune.RunConfig(progress_reporter=reporter),
    )

    results = tuner.fit()
    best = results.get_best_result(metric="accuracy", mode="max")

    print("Best config:", best.config)

    best_config = best.config

# Train the Model

In [ ]:
if not RUN_RAYTUNE:
    model, optimizer, criterion = build_model(1e-3, num_classes)
    
    epochs = 5

    for epoch in range(epochs):
        print(f"\n--- Epoch {epoch+1}/{epochs} ---")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc     = evaluate(model, val_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

# Test the Model

In [ ]:
if not RUN_RAYTUNE:
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)

    print("\n=== Test Results ===")
    print(f"Test Loss:     {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")